Установка необходимых библиотек


In [ ]:
!pip install fasttext
!pip install tqdm

Монтирование диска Google Drive для загрузк и харанения  данных.

**ВНИМАНИЕ** для доступа к данным необходимо выполнить следующие действия:

1. Перейти по [ссылке](https://drive.google.com/drive/folders/1MnoxVXjG8o8kL0SS14E3YzbZmCovsGpR?usp=sharing)
2. Создать ярлык на данную директорию в своём Google Drive. Для этого нажмите на название директории а затем на "Добавить ярлык на Диск".

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Импорт необходимых компонентов

In [ ]:
from fasttext import FastText
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import xml.etree.ElementTree as Etree
import requests
import regex as re
from random import randint, random

prefix = Path("/content/gdrive/My Drive/Kononov_NLP")

Выбор и загрузка предобученной модели для генерации векторного представления слов

In [ ]:
# name, alias = "ft_native_300_ru_wiki_lenta_lower_case", "wiki"
# name, alias = "ft_native_300_ru_twitter_nltk_word_tokenize", "twitter"
name, alias = "cc.ru.300", "original"
model = FastText.load_model(str(prefix / "FastText/{}.bin".format(name)))

Генерация синонимичных слов состоит из двух частей:
1. Генерация на основе ближайших соседей слова в пространстве эмбеддингов
2. Генерация на основе словаря Yandex Dictionary

Создание корпуса - множества всех слов, входящих в запросы

In [ ]:
queries_df = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)

corpus = set()
for query in queries_df["text"]:
  corpus.update(query.split())

---

**1. Генерация на основе ближайших соседей слова в пространстве 
эмбеддингов**

Установка порога, слова, имеющие похожесть ниже которого нас не интересуют

In [ ]:
threshold = 0.8

In [ ]:
output_path = prefix / "simialr_queries_words_nn.tsv"
with open(output_path, "w") as output_stream:
  output_stream.write("word_a\tword_b\tscore\n")  
  for word in tqdm(corpus, position=0, leave=True):
    sinonims = model.get_nearest_neighbors(word, k=5)
    for item in sinonims: 
      if item[0] > threshold:
        output_stream.write("{}\t{}\t{:05.3f}\n".format(word, item[1], item[0]))

---

**2. Генерация на основе синонимов из Yandex Dictionary**

Функция добавьте сгенерованные на [сайте](https://yandex.ru/dev/dictionary/) api ключи в переменную api_keys.

In [ ]:
url_base = "https://dictionary.yandex.net/api/v1/dicservice/lookup?key={}&lang=ru-en&text={}"
api_key= ["dict.1.1.20200614T093641Z.6ef4f758e1bde788.6bcb115395a67b382dbbf07cf9e017f0562f5da4",
          "dict.1.1.20200614T093949Z.1159baedcb8fba61.da66fba1ca1dc871a4b99d6d70ea322ded9fa130",
          "dict.1.1.20200614T094544Z.28e27a08835d6ea1.8b417d5ae635087b9ebd6c72aa9eec85733d6f2c"]
OK_RESPONSE_CODE = 200

Функция отправки запросов и обработки результатов

In [ ]:
def send_request(self, word):
  idx = randint(0, len(self.api_key) - 1)
  url = self.url_base.format(self.api_key[idx], word)
  try:
    response = requests.get(url)
  except Exception as err:
    print(err)
    return ["ERROR", ]
  if response.status_code != self.OK_RESPONSE_CODE:
    print(response.text)
    print("Warning: response status code {}".format(response.status_code))
    header = "; ".join([str(key) + " : " + str(value) for key, value in response.headers.items()])
    print("Response header: \n{}".format(header))
    return ["BAD_CODE", ]
  root = Etree.fromstring(response.text)
  result = []
  if len(root) > 1:
    for i, spellResult in enumerate(root[1].findall("*.mean")):
      result.append(spellResult[0].text)
    return result
  else:
    return ["NONE", ]

Генерация синонимов для всех слов корпуса

In [ ]:
dictionary = dict()
for word in tqdm(corpus):
    syns = self._send_request(word)
  if syns:
    dictionary[word] = syns

Подсчёт похожести для каждой пары синонимов

In [ ]:
data = []
for key, value in syn.items():
    data += [(key, x) for x in value]
df = pd.DataFrame(data, columns=["word_a", "word_b"])
df = df[df["word_b"] != "NONE"]
df = df[df["word_b"] != "ERROR"]
df = df.assign(score=np.zeros(syns.index.size, dtype=float))

for key, group in tqdm(syns.groupby("word_a")):
  word_emb = model.get_word_vector(key).reshape(1, -1)
  words_emb = np.array(group["word_b"].apply(model.get_sentence_vector).to_list())
  syns.loc[group.index, "score"] = cosine_similarity(word_emb, words_emb).reshape(-1)

Отделение синонимов похожесть которых превышает заданный порог и cохранение результатов

In [ ]:
threshold = 0.3

df = df[df["score"] > threshold]
df = df.sort_values("word_a")

df.to_csv(prefix / "simialr_queries_words_ya.tsv", sep="\t", index=False)

---

**Генерация похожих запросов**

In [ ]:
queries = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)

nn_df = pd.read_csv(prefix / "simialr_queries_words_nn.tsv", sep="\t")
ya_df = pd.read_csv(prefix / "simialr_queries_words_ya.tsv", sep="\t")
similarity_df = pd.concat([nn_df, ya_df], ignore_index=True)

similarity_df["word_a"] = similarity_df["word_a"].apply(str.lower)
similarity_df["word_b"] = similarity_df["word_b"].apply(str.lower).apply(lambda x: re.sub(r'[^\w\s]', '', x))
similarity_df.drop_duplicates(subset=["word_a", "word_b"], inplace=True)
similarity_df = similarity_df[similarity_df["word_a"] != similarity_df["word_b"]]
similarity_df = similarity_df[similarity_df["word_a"].str.isalpha()]

similar_groups = similarity_df.groupby("word_a")


with open(prefix / "wider_queries.tsv", "w") as output_stream:
    valid_set = set(similarity_df["word_a"].unique())
    for key, row in tqdm(list(queries.iterrows())):
        words = row["text"].split()
        new_queries = set()
        new_queries.add(tuple([row["text"],  1]))
        for i in range(20):
            new_query = []
            score = 1
            for word in words:
                if word in valid_set:
                    sim_words = similar_groups.get_group(word)
                    idx = randint(0, sim_words.index.size - 1)
                    sim_word = sim_words["word_b"].values[idx]
                    if random() < 0.3:
                        score *= sim_words["similarity"].values[idx]
                        new_query.append(sim_word)
                    else:
                        new_query.append(word)
                else:
                    new_query.append(word)
            new_queries.add(tuple([" ".join(new_query), score]))
        for item in new_queries:
            output_stream.write("{}\t{}\t{:04.2f}\n".format(key, item[0], item[1]))

    raw_queries = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)
    for key, row in raw_queries.iterrows():
        output_stream.write("{}\t{}\t1.00\n".format(key, row["text"]))

df = pd.read_csv(prefix / "wider_queries.tsv", sep="\t", index_col=0, names=["text", "score"])
df = df.drop_duplicates(subset=["text"]).sort_index()
df.to_csv(prefix / "wider_queries.tsv", sep="\t", index_label="query_id")
